## Quest
- 머신러닝을 이용한 감성분석
- 영화 댓글 학습(0:부정, 1:긍정)
- 타 사이트 댓글 이용 성능 확인

In [ ]:
%pip install wget

In [1]:
import wget

In [ ]:
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

In [2]:
import pandas as pd
df_naver_movie_comments = pd.read_csv('./ratings_train.txt', sep='\t')
df_naver_movie_comments[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
df_naver_movie_comments = df_naver_movie_comments[:1000]

In [4]:
df_nagative = df_naver_movie_comments.loc[df_naver_movie_comments['label'] == 0]
df_nagative

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
...,...,...,...
993,9673323,마이클베이 최근작을 볼때마다 느낀다. 블록버스터 액션장르라도 탄탄한 시나리오는 영화...,0
994,8121444,0점은없나?1점은 너무 후하네,0
995,10275296,미달이는.. 연예계를 바라보는 태도를 고쳐야 한다.,0
996,6473973,파괴된 관객들.,0


In [5]:
df_positive = df_naver_movie_comments.loc[df_naver_movie_comments['label'] == 1]
df_positive

,id,document,label
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1
10,9008700,걍인피니트가짱이다.진짜짱이다♥,1
...,...,...,...
990,474835,마지막이 좀황당했지만 좋은영화,1
991,3279841,우연히 tv를 돌려보다가 사로잡혔다. 최근 10년간 본 코미디 영화 중 최고,1
992,8407277,연 기 굿 화이팅해요누나,1
997,8417568,알콜중독자들 덕에 평점이 낮은가?,1


In [3]:
negative_list = list(df_nagative['document'])

NameError: name 'df_nagative' is not defined

In [4]:
positive_list = list(df_positive['document'])

NameError: name 'df_positive' is not defined

In [5]:
comments_list = negative_list + positive_list

NameError: name 'negative_list' is not defined

### 불용어 처리

In [6]:
# !pip install python-mecab-ko

In [1]:
from mecab import MeCab
mecab = MeCab()

# 품사 달기
mecab.pos(comments_list[0])

NameError: name 'comments_list' is not defined

### 형태소 분석기를 사용하여 품사 분류

In [ ]:
tokenized_comments = list()
for comment in comments_list:
    comment_morphs = mecab.morphs(comment)
    
    # 불용어 처리
    
    
    tokenized_comments.append(' '.join(comment_morphs))
tokenized_comments

### 벡터 토큰화(단어 숫자화)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
countVectorizer = CountVectorizer()
countVectorizer.fit(comments)

In [ ]:
countVectorizer.vocabulary_

In [ ]:
tfidfVectorizer = TfidfVectorizer(use_idf=True)
tfidfVectorizer.fit(comments)

In [ ]:
tfidfVectorizer.idf_

In [ ]:
features = tfidfVectorizer.transform(comments)

### 목표변수 생성

In [ ]:
target = ['긍정'] * len(comments_positive) + ['부정'] * len(comments_negative)
target

### 머신러닝 학습

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC()
model.fit(features.toarray(), target)

### 머신러닝을 통한 에측
- 다음사이트 '영화' 댓글을 이용해 예측

In [ ]:
# comment = '놀란의 영화는 정말 미쳤다는 말밖에 안 나온다. 광복절을 맞이하여 이런 영화를 선사해준 놀란에게 경의를 표합니다.'

In [ ]:
# tokenized_comment = mecab.morphs(comment)

In [ ]:
# vectorized_comment = tfidfVectorizer.transform([' '.join(tokenized_comment)]) # 2차원으로 넣어줌
# vectorized_comment.toarray()  #벡터화

In [ ]:
# model.predict(vectorized_comment.toarray())